# Demo of ```neuralg``` module

## Spectral graph theory application

In spectral graph theory, graph properties are studied in light of the eigendecompositions of their associated matrices. For instance, graph characteristics can be inferred from the eigenvalue of the adjacency matrix $A$ of a graph, i.e 
$$ A_{ij} = \begin{cases} 1 \; \text{if there is an edge from $i$ to $j$} \\ 0 \; \text{otherwise} \end{cases}$$
A more detailed read can be found e.g. [here](https://en.wikipedia.org/wiki/Spectral_graph_theory).
## A machine learning task using the neuralg module
To carry out automatic differentation to allow, for example, gradient-based optimization, the eigenvalue approximations must not break the gradient flow. Training a neural network with PyTorch will serve as a demonstration that the `neuralg` module indeed preserves gradients. Lets say we want a neural network that approximates the total number of length $k$ cycles starting from any node in a graph. In exact arithmetic, this quantity is given by 

$$trace(A^k),$$

where $A$ is the adjacency matrix of the graph, but we do not want to perform the $k$ matrix multiplications. From the properties of the trace operator, we can relate this quantity to  the spectrum of $A$ such that  

\begin{equation} \# \text{ cycles of length } k = \sum_{i} \lambda_i(A)^k 
\end{equation}


To this end, we can use `neuralg.eigvals()` to approximate the ground truths in the supervised learning of predicting this quantity.  

In [1]:
import neuralg 
from neuralg.evaluation.compute_accuracy import compute_accuracy
from neuralg.training.losses import relative_L1_evaluation_error
import torch
import networkx # additional requirement for processing graphs 
from copy import deepcopy
import time
import matplotlib.pyplot as plt

12:02:23|neuralg-INFO| Initialized neuralg for cpu
12:02:23|neuralg-INFO| Setting Torch's default tensor type to Float32 (CUDA not initialized).


### `neuralg` allows for setting float precision and can potentially be employed on a GPU, if available.

In [2]:
neuralg.set_up_torch(torch_enable_cuda=True)
neuralg.set_precision("float64")

12:02:27|neuralg-WARNING| Error enabling CUDA. cuda.is_available() returned False. CPU will be used.
12:02:27|neuralg-INFO| Setting Torch's default tensor type to Float64 (CUDA not initialized).


### Define a simple convolutional net for the regression task. A convolutional block is followed by two dense layers.

In [3]:
import torch.nn as nn 
import torch.nn.functional as F

class CycleCNN(nn.Module):
    def __init__(self, n_graph_nodes, conv_layers, filters, kernel_size):
        super(CycleCNN, self).__init__()
        self.net = []
        self.n_graph_nodes = n_graph_nodes
        self.net.append(nn.Conv2d(1,filters,kernel_size, padding = "same"))
        self.net.append(nn.BatchNorm2d(filters))
        self.net.append(nn.ReLU())
        for i in range(conv_layers-1):
            self.net.append(nn.Conv2d(filters,filters,kernel_size, padding = "same"))
            self.net.append(nn.BatchNorm2d(filters))
            self.net.append(nn.ReLU())
        
        self.net.append(nn.Conv2d(filters,1,kernel_size, padding = "same"))
        self.net.append(nn.Flatten())
        self.net.append(DenseLayer(n_graph_nodes**2,n_graph_nodes))
        self.net.append(DenseLayer(n_graph_nodes,1, is_last = True))
        self.net = nn.Sequential(*self.net)
    
    def forward(self, x):
        out = self.net(x)
        return out

class DenseLayer(nn.Module):

    def __init__(self, in_features, out_features, bias=True, is_last = False):
        super().__init__()
        self.is_last = is_last
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, input):
        if self.is_last: 
            return self.linear(input)
        else:
            return F.relu(self.linear(input))

#### Define graph generation properties and target cycle length and necessary training parameters

In [4]:
#Data parameters 
n_graph_nodes = 5 # Number of nodes in the graph, defining the size of the adjacancy matrices
k = 5 # Cycle length of interest
p = 0.5 # Probability of two nodes having a connecting edge 


#Training parameters
iterations = 2000 # Iterations per epoch
batch_size = 32 # Number of matrices per forward pass
epochs = 5
criterion = nn.MSELoss() # We use a mean square error loss 

In [5]:
def get_adjacency_batch(batch_size,n_graph_nodes, fixed_seed = False):
    """ Simple generator of random graph adjacency matrices.       
    Args:
        batch_size (int): Size of batch to be generated
        n_graph_nodes (int): Corresponds to size of the generaed adjacency matrices. 
        fixed_seed (bool, optional): If true, the seed is fixed in the sampling. Defaults to False.

    Returns:
        tensor: batch of adjacency matrices, of shape [batchsize,1,n_graph_nodes,n_graph_nodes] 
    """
    if fixed_seed:
        torch.manual_seed(1)
    for i in range(batch_size):
        g = networkx.erdos_renyi_graph(n=n_graph_nodes, p= 0.5)
        adj_matrix = torch.tensor(networkx.to_numpy_array(g))[None,:]
        if i == 0: 
            A = adj_matrix
        else:   
            A = torch.cat((A,adj_matrix))
        
    return A[:,None,:] # broadcasts to an extra channel dimension 

In [6]:
def train(model: nn.Module, use_neuralg=True) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 1000
    start_time = time.time()

    for i in range(iterations):

        A = get_adjacency_batch(batch_size, n_graph_nodes=n_graph_nodes) #Sample a batch

        output = model(A) #Predict # of k-cycles with model

    
        if use_neuralg:
            target_eigvals = neuralg.eigvals(A, symmetric = True) #Use neuralg module to compute ground truth
        else:
            target_eigvals = torch.linalg.eigvalsh(A) #Or, use torch built-in 
        
        target = torch.pow(target_eigvals,k).sum(-1) #The target is the quantiti in Eq.(1)
        
        loss = criterion(output,target) # Compute loss
    
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        total_loss += loss.item()
        
        if i % log_interval == 0 and i > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval

            print(f'| epoch {epoch:3d} | {i:5d} batches | '
                  f'lr {lr:02.4f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.5f}')

            total_loss = 0
            start_time = time.time()
    return ms_per_batch
    

### Training a network 
##### For reference, two identically initialized models are trained using either `neuralg.eig()` or `torch.linalg.eigvalsh()` to compute the eigenvalue ground truths. The loss is computed between the model outputs and the target function of the eigenvalues from Eq.(1), i.e. the analytical solution to number of total paths of length $k$. 

In [7]:
trained_models = []
for use_neuralg in [True,False]:
    model = CycleCNN(n_graph_nodes, conv_layers = 3,filters = 32,kernel_size=3) #Instantiate model
    lr = 3e-4 # learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
    model.train(); # turn on train modeepochs = 1
    
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        ms_per_batch = train(model,use_neuralg)
        scheduler.step()
    trained_models.append([deepcopy(model),ms_per_batch])

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


| epoch   1 |  1000 batches | lr 0.0003 | ms/batch 14.53 | loss 4937.77547


KeyboardInterrupt: 

#### Evaluation from training
When evaluating on the test set, we consider a predicted output $\hat{y}$ to be a correct solution to the problem if it approximates the ground truth $y$ to a given tolerance $\tau$. Specifically, we check that $|y-\hat{y}| \leq \tau|y|$ is satisfied. For completeness, the test set ground truths are calculated both using `neuralg.eig()` and `torch.eigvalsh()`.

In [ ]:
eval_size = 10000 # Size of evaluation set 
eval_set = get_adjacency_batch(eval_size, n_graph_nodes, fixed_seed=True) #Sample a test set

#Since path length should be an integer, we round the final predictions with the two trained models
neuralg_pred = torch.round(trained_models[0][0](eval_set)) # Model trained with neuralg ground truths
torch_pred = torch.round(trained_models[1][0](eval_set)) # Model trained with torch ground truths

torch_targets = torch.round(torch.pow(torch.linalg.eigvalsh(eval_set),k).sum(-1)) # We evaluation eigenvalues with torch built-int
neuralg_targets = torch.round(torch.pow(neuralg.eig(eval_set),k).sum(-1))

#Model trained with neuralg 
neuralg_to_neuralg_eval_errors= relative_L1_evaluation_error(neuralg_pred,neuralg_targets)
neuralg_to_torch_eval_errors= relative_L1_evaluation_error(neuralg_pred,torch_targets)

#Model trained with torch
torch_to_neuralg_eval_errors = relative_L1_evaluation_error(torch_pred,neuralg_targets) 
torch_to_torch_eval_errors = relative_L1_evaluation_error(torch_pred,torch_targets) 

tolerances = [0.1,0.08,0.06,0.05,0.025,0.02,0.01,0.005,0.0001]
n_neuralg_acc = []
n_torch_acc = []
t_neuralg_acc = []
t_torch_acc = []
for tol in tolerances:
    n_neuralg_acc.append(compute_accuracy(tol,neuralg_to_neuralg_eval_errors).item())
    n_torch_acc.append(compute_accuracy(tol,neuralg_to_torch_eval_errors).item())
    t_neuralg_acc.append(compute_accuracy(tol,torch_to_neuralg_eval_errors).item())
    t_torch_acc.append(compute_accuracy(tol,torch_to_torch_eval_errors).item())

In [ ]:
fig = plt.figure(figsize=(14, 6), dpi=150)
fig.patch.set_facecolor("white")
ax = fig.add_subplot(1,2,1)
ax.set_title("neuralg trained, {:.3} ms per batch. \n Test accuracy versus tolerance, {}x{}".format(trained_models[0][1],n_graph_nodes, n_graph_nodes), fontsize=18)
ax.plot(tolerances,n_neuralg_acc,label="neuralg evaluation ground truths")
ax.plot(tolerances,n_torch_acc,label="torch evaluation ground truths")

ax.legend(fontsize=14)
ax.set_xlabel("$\\tau $", fontsize=18)
ax.set_ylabel("Accuracy", fontsize=16);

ax = fig.add_subplot(1,2,2)
ax.set_title("torch trained, {:.3} ms per batch.\n Test accuracy versus tolerance, {}x{}".format(trained_models[1][1],n_graph_nodes, n_graph_nodes), fontsize=18)
ax.plot(tolerances,t_neuralg_acc,label="neuralg evaluation ground truths")
ax.plot(tolerances,t_torch_acc,label="torch evaluation ground truths")

ax.legend(fontsize=14)
ax.set_xlabel("$\\tau $", fontsize=18)
ax.set_ylabel("Accuracy", fontsize=16);

### Conclusion 
- In evaluation, the model trained with `neuralg` eigenvalues performs well also on the more accurate torch evaluatiom ground truths. 
- Training with `neuralg` or `torch` modules seem to yield similarly performing models. 
- When run On CPU there is no synchronization needed and for these small matrices, the computational cost of the numerical methods is very small.  For larger matrices and on the GPU, we expect `neuralg`to have an edge. 

#### After using the module, we can clear the loaded models to free the allocated memory

In [ ]:
neuralg.clear_loaded_models()
assert neuralg.neuralg_ModelHandler.loaded_models == {}